In [1]:
# Score and Scale Calculation Functions (matching viphl_strategy_scoring.py)

def calculate_hl_byp_score(m, n, k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=False):
    '''
    Calculate normalized HL byP score (0-1) based on m,n parameters with power scaling
    
    Args:
        m, n: pivot point window parameters (leftbars, rightbars)
        k: power_scaling_factor for window size calculation
        max_mn_cap: maximum cap for normalization
        by_point_weight: base weight for pivot points
        on_trend_ratio: multiplier for trending conditions
        is_trending: whether this is a trending condition
    
    Returns:
        final_score: normalized score in range [0, 1]
    '''
    # Normalize window size to 0-1 range using power scaling: (m**k + n**k) / (2 * max_mn_cap**k)
    window_score = min((m**k + n**k) / (2 * (max_mn_cap ** k)), 1.0)
    
    # Apply weight multipliers and condition-specific normalization
    if is_trending:
        # Trending conditions are more significant with on_trend_ratio multiplier
        weight_multiplier = by_point_weight * on_trend_ratio
        max_possible_weight = by_point_weight * on_trend_ratio  # Trending-specific max
    else:
        # Normal conditions use base weight
        weight_multiplier = by_point_weight
        max_possible_weight = by_point_weight  # Normal-specific max
    
    # Final score incorporating weights (normalized to each condition's max)
    final_score = min(window_score * weight_multiplier / max_possible_weight, 1.0)
    
    return final_score


def calculate_combined_score(high_score, low_score, high_score_scaling_factor, low_score_scaling_factor):
    '''
    Combine high and low scores with scaling factors
    
    Args:
        high_score: score from high pivot point
        low_score: score from low pivot point
        high_score_scaling_factor: weight for high pivot contribution
        low_score_scaling_factor: weight for low pivot contribution
    
    Returns:
        combined_score: weighted average in range [0, 1]
    '''
    weighted_high = high_score * high_score_scaling_factor
    weighted_low = low_score * low_score_scaling_factor
    total_weight = high_score_scaling_factor + low_score_scaling_factor
    combined_score = (weighted_high + weighted_low) / total_weight
    
    return combined_score


def calculate_pnl_scale(combined_score):
    '''
    Maps combined_score (0-1) to PnL scale (1-3)
    
    Current: Linear relationship
    Formula: scale = 1 + 2 * score
    
    Characteristics:
    - Linear/proportional relationship between score and scale
    - Constant growth rate across all score ranges
    - Balanced reward structure
    
    Examples:
    - score=0.0 → scale=1.00 (minimum)
    - score=0.5 → scale=2.00 (midpoint)
    - score=1.0 → scale=3.00 (maximum)
    '''
    # Linear scaling (current)
    scale = 1.0 + 2.0 * combined_score
    
    # Alternative: Square root scaling (commented out)
    # scale = 1.0 + 2.0 * (combined_score ** 0.5)
    # Examples: score=0.1→1.63, score=0.5→2.41, score=1.0→3.00
    
    return scale

# VipHL Strategy Scoring System

This notebook demonstrates the score and scale calculation logic from `viphl_strategy_scoring.py`.

## Complete Scoring Flow

1. **HL byP Score Calculation** (`calculate_hl_byp_score`)
   - Input: `m, n` (pivot window parameters), `k` (power scaling factor), `max_mn_cap` (normalization cap)
   - Formula: `window_score = min((m^k + n^k) / (2 * max_mn_cap^k), 1.0)`
   - Apply weight multipliers based on trending vs normal conditions
   - Output: `final_score` in range [0, 1]

2. **Combined Score** (`calculate_combined_score`)
   - Combine high and low pivot scores with scaling factors
   - Formula: `combined_score = (high_score * high_weight + low_score * low_weight) / total_weight`
   - Output: `combined_score` in range [0, 1]

3. **PnL Scale** (`calculate_pnl_scale`)
   - **Current: Linear relationship**
   - Formula: `scale = 1 + 2 * combined_score`
   - Maps score [0, 1] to scale [1.0, 3.0]
   - This scale is **directly multiplied** with percentage returns for all PnL calculations

## Default Parameters (from run configuration)

- `max_mn_cap = 12`
- `power_scaling_factor (k) = 1.0` (linear)
- `high_score_scaling_factor = 0.5`
- `low_score_scaling_factor = 0.5`
- `by_point_weight = 1`
- `on_trend_ratio = 1` (no boost for trending)

## Examples

Run the cells below to see how different parameters affect the final PnL scale.

In [8]:
# Test with default parameters from viphl_strategy_scoring.py
# Default configuration values
k = 1.1              # power_scaling_factor
max_mn_cap = 20      # max_mn_cap (from run configuration)
by_point_weight = 1  # base weight
on_trend_ratio = 1   # on_trend_ratio (from run configuration)


a = 14
# Test parameters for high pivot (normal condition)
m_high = a  # high_by_point_n
n_high = a  # high_by_point_m

# Test parameters for low pivot (normal condition)
m_low = a  # low_by_point_n
n_low = a  # low_by_point_m

# Calculate scores for normal (non-trending) conditions
high_score = calculate_hl_byp_score(m_high, n_high, k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=False)
low_score = calculate_hl_byp_score(m_low, n_low, k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=False)

print(f"High Score (m={m_high}, n={n_high}, normal): {high_score:.4f}")
print(f"Low Score (m={m_low}, n={n_low}, normal): {low_score:.4f}")

# Combine scores with scaling factors
high_score_scaling_factor = 0.5  # from run configuration
low_score_scaling_factor = 0.5   # from run configuration

combined_score = calculate_combined_score(high_score, low_score, high_score_scaling_factor, low_score_scaling_factor)
print(f"\nCombined Score: {combined_score:.4f}")

# Calculate PnL scale
pnl_scale = calculate_pnl_scale(combined_score)
print(f"PnL Scale: {pnl_scale:.4f}")
print(f"\nThis means PnL will be multiplied by {pnl_scale:.2f}x")

High Score (m=14, n=14, normal): 0.6755
Low Score (m=14, n=14, normal): 0.6755

Combined Score: 0.6755
PnL Scale: 2.3509

This means PnL will be multiplied by 2.35x


In [ ]:
# Test with trending conditions
print("=== Testing Trending vs Normal Conditions ===\n")

# Test parameters for trending conditions (from strategy defaults)
m_high_trend = 10  # high_by_point_n_on_trend
n_high_trend = 10  # high_by_point_m_on_trend
m_low_trend = 10   # low_by_point_n_on_trend
n_low_trend = 10   # low_by_point_m_on_trend

# Calculate scores for trending conditions
high_score_trend = calculate_hl_byp_score(m_high_trend, n_high_trend, k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=True)
low_score_trend = calculate_hl_byp_score(m_low_trend, n_low_trend, k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=True)

print(f"TRENDING High Score (m={m_high_trend}, n={n_high_trend}): {high_score_trend:.4f}")
print(f"TRENDING Low Score (m={m_low_trend}, n={n_low_trend}): {low_score_trend:.4f}")

# Combine and calculate scale
combined_score_trend = calculate_combined_score(high_score_trend, low_score_trend, high_score_scaling_factor, low_score_scaling_factor)
pnl_scale_trend = calculate_pnl_scale(combined_score_trend)

print(f"TRENDING Combined Score: {combined_score_trend:.4f}")
print(f"TRENDING PnL Scale: {pnl_scale_trend:.4f}")

print("\n--- Comparison ---")
print(f"Normal PnL Scale: {pnl_scale:.4f}x")
print(f"Trending PnL Scale: {pnl_scale_trend:.4f}x")
print(f"Difference: {pnl_scale_trend - pnl_scale:.4f}x ({((pnl_scale_trend/pnl_scale - 1)*100):.2f}% increase)")

In [ ]:
# Test how PnL scale changes with different window sizes (m=n)
print("=== PnL Scale vs Window Size (m=n, Normal Conditions) ===\n")
print(f"Parameters: k={k}, max_mn_cap={max_mn_cap}, high_weight={high_score_scaling_factor}, low_weight={low_score_scaling_factor}\n")
print("m=n | High Score | Low Score | Combined | PnL Scale")
print("-" * 60)

for i in range(4, 21):
    m = n = i
    
    # Calculate scores for both high and low (assuming same m,n)
    high_score_i = calculate_hl_byp_score(m, n, k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=False)
    low_score_i = calculate_hl_byp_score(m, n, k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=False)
    
    # Combine scores
    combined_score_i = calculate_combined_score(high_score_i, low_score_i, high_score_scaling_factor, low_score_scaling_factor)
    
    # Calculate PnL scale
    pnl_scale_i = calculate_pnl_scale(combined_score_i)
    
    print(f"{i:3d} | {high_score_i:10.4f} | {low_score_i:9.4f} | {combined_score_i:8.4f} | {pnl_scale_i:9.4f}")

print("\n" + "=" * 60)
print(f"At max_mn_cap={max_mn_cap}: combined_score={combined_score_i:.4f}, scale={pnl_scale_i:.4f}x")
print(f"Maximum possible scale: 3.00x (when combined_score=1.0)")

In [ ]:
# Test effect of different power_scaling_factor (k) values
print("=== Impact of Power Scaling Factor (k) ===\n")
print("Testing with m=n=10, max_mn_cap=12\n")
print("k value | Window Score | Combined Score | PnL Scale")
print("-" * 60)

m = n = 10
test_k_values = [0.5, 0.8, 1.0, 1.2, 1.5, 2.0]

for test_k in test_k_values:
    # Calculate with different k values
    high_score_k = calculate_hl_byp_score(m, n, test_k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=False)
    low_score_k = calculate_hl_byp_score(m, n, test_k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=False)
    
    combined_score_k = calculate_combined_score(high_score_k, low_score_k, high_score_scaling_factor, low_score_scaling_factor)
    pnl_scale_k = calculate_pnl_scale(combined_score_k)
    
    # Window score is the raw score before weighting
    window_score = min((m**test_k + n**test_k) / (2 * (max_mn_cap ** test_k)), 1.0)
    
    print(f"{test_k:7.1f} | {window_score:12.4f} | {combined_score_k:14.4f} | {pnl_scale_k:9.4f}x")

print("\nNote:")
print("- k < 1.0: Diminishing returns (slower growth)")
print("- k = 1.0: Linear relationship")
print("- k > 1.0: Exponential growth (rewards larger windows more)")

In [ ]:
# Complete example: Different high/low window sizes showing weighting effect
print("=== Complete Example: Different High/Low Window Sizes ===\n")

# Scenario 1: Larger high window, smaller low window
print("Scenario 1: High window (m=12, n=12), Low window (m=8, n=8)")
print("-" * 60)

m_high_1 = 12
n_high_1 = 12
m_low_1 = 8
n_low_1 = 8

high_score_1 = calculate_hl_byp_score(m_high_1, n_high_1, k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=False)
low_score_1 = calculate_hl_byp_score(m_low_1, n_low_1, k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=False)

print(f"High Score (m={m_high_1}, n={n_high_1}): {high_score_1:.4f}")
print(f"Low Score (m={m_low_1}, n={n_low_1}): {low_score_1:.4f}")

# Test with different weighting
for high_w, low_w in [(0.5, 0.5), (0.7, 0.3), (0.3, 0.7)]:
    cs = calculate_combined_score(high_score_1, low_score_1, high_w, low_w)
    ps = calculate_pnl_scale(cs)
    print(f"  Weights ({high_w:.1f}, {low_w:.1f}): Combined={cs:.4f}, Scale={ps:.4f}x")

print("\n" + "=" * 60)

# Scenario 2: Equal windows
print("\nScenario 2: Equal windows (m=10, n=10)")
print("-" * 60)

m_high_2 = 10
n_high_2 = 10
m_low_2 = 10
n_low_2 = 10

high_score_2 = calculate_hl_byp_score(m_high_2, n_high_2, k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=False)
low_score_2 = calculate_hl_byp_score(m_low_2, n_low_2, k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=False)

print(f"High Score (m={m_high_2}, n={n_high_2}): {high_score_2:.4f}")
print(f"Low Score (m={m_low_2}, n={n_low_2}): {low_score_2:.4f}")

cs_2 = calculate_combined_score(high_score_2, low_score_2, 0.5, 0.5)
ps_2 = calculate_pnl_scale(cs_2)
print(f"Combined Score: {cs_2:.4f}")
print(f"PnL Scale: {ps_2:.4f}x")

print("\n" + "=" * 60)
print("Key Insight: Weighting allows emphasizing high or low pivot quality when they differ.")

In [ ]:
# Test effect of different high/low score weighting
print("=== Impact of High/Low Score Weighting ===\n")
print("Testing with m_high=n_high=10, m_low=n_low=10\n")
print("High Weight | Low Weight | Combined Score | PnL Scale")
print("-" * 60)

m = n = 10
weight_combinations = [
    (1.0, 0.0),  # Only high matters
    (0.75, 0.25),
    (0.5, 0.5),  # Equal weight (current default)
    (0.25, 0.75),
    (0.0, 1.0),  # Only low matters
]

for high_weight, low_weight in weight_combinations:
    # Calculate individual scores
    hs = calculate_hl_byp_score(m, n, k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=False)
    ls = calculate_hl_byp_score(m, n, k, max_mn_cap, by_point_weight, on_trend_ratio, is_trending=False)
    
    # Combine with different weights
    cs = calculate_combined_score(hs, ls, high_weight, low_weight) if (high_weight + low_weight) > 0 else 0.0
    ps = calculate_pnl_scale(cs)
    
    print(f"{high_weight:11.2f} | {low_weight:10.2f} | {cs:14.4f} | {ps:9.4f}x")

print("\nNote: With equal window sizes (m=n), weighting has no effect.")
print("Weighting matters when high and low pivots have different m,n values.")